In [ ]:
import tensorflow as tf

import keras

import h5py

# IMPORTANDO COMANDOS E FUNÇÕES #

import numpy as np
import matplotlib.pyplot as plt #gerar gráfico
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

import pickle

# IMPORTANDO LAYERS PARA A CNN #

from keras.models import load_model
from keras import applications
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D

In [ ]:
# DIVIDIR AS 8000 IMAGENS EM PARTIÇÃO DE TREINO E VALIDAÇÃO, UTILIZANDO O IMAGEDATAGENERATOR #

from keras.preprocessing.image import ImageDataGenerator

datagen =  ImageDataGenerator(rescale = 1./255, validation_split=0.2) # comando para reescala + divisão em 20% para validação

# imagens de treino (80%) 

train_data = datagen.flow_from_directory('outputs/output_train_val', 
                                         subset = 'training',
                                         batch_size = 64,
                                         target_size = (224, 224)) # definindo tamanho das imagens

# imagens de validação (20%)

valid_data = datagen.flow_from_directory('outputs/output_train_val',
                                         subset='validation',
                                         batch_size = 64,
                                         target_size = (224, 224))

In [ ]:
datagen_test =  ImageDataGenerator(rescale = 1./255) # comando para reescala das imagens de teste

# IMAGENS DE TESTE

test_data = datagen_test.flow_from_directory(directory='outputs/output_test',
                                             target_size = (224, 224),
                                             batch_size = 1,
                                             shuffle=False)

In [ ]:
# acessando o modelo pré treinado ResNet-50 #

base_model = keras.applications.resnet50.ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3))

In [ ]:
# adicionando pooling global de média + camada de dropout #

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)

# adicionando uma camada totalmente conectada e a camada de saída #

x = Dense(1024, activation='relu')(x)

predictions = Dense(11, activation='softmax')(x)

# por fim, o modelo que será treinado: #

model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
# apenas as últimas camadas serão treinadas (as inicializadaasc com pesos aleatórios) #
# para tal, congelam-se as camadas já treinadas da ResNet-50 #

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# compilando o modelo #

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# DEFININDO O COMANDO EARLYSTOPPING #

es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience = 7) #early stopping baseado na melhor acurácia de validação

In [ ]:
# INÍCIO DO TREINO DO MODELO (em novos dados, por poucas épocas) #

model.fit(train_data,
          epochs=100,
          validation_data=valid_data,
          callbacks=[es])

# informação: steps_per_epoch = número de imagens / batch_size

In [ ]:
# após isso, as últimas camadas já estão treinadas, podemos iniciar o ajuste fino #
# das camadas convolutivas da ResNet-50, congelando camadas iniciais #
# e treinando as camadas restantes #

In [ ]:
# visualizar o nome/indíces das camadas, para saber quais congelar #

for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

In [ ]:
# exemplo: congelar primeiras 249 camadas e descongelar o resto #

for layer in model.layers[:249]:
    layer.trainable = False

for layer in model.layers[249:]:
    layer.trainable = True

In [ ]:
# recompilando o modelo para que as alterações façam efeito #

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# re-treinamento do modelo #

history = model.fit(train_data,
                    epochs=100,
                    validation_data=valid_data,
                    callbacks=[es])

In [ ]:
model.save('1teste_ResNet50_finetuning',
          overwrite = True) 

with open('trainHistoryDict_1teste_ResNet50_finetuning', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

In [ ]:
model = keras.models.load_model('1teste_ResNet50_finetuning')
history = pickle.load(open('trainHistoryDict_1teste_ResNet50_finetuning', "rb"))

In [ ]:
# GERANDO GRÁFICO COM RESULTADOS DAS ACURÁCIAS DE TREINO E VALIDAÇÃO #

plt.plot(history['accuracy'], label='accuracy_train')
plt.plot(history['val_accuracy'], label = 'accuracy_valid')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [ ]:
# GERANDO GRÁFICO COM RESULTADOS DO LOSS DE TREINO E VALIDAÇÃO #

plt.plot(history['loss'], label='loss train')
plt.plot(history['val_loss'], label='loss val')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 3])
plt.legend(loc='lower right')

In [ ]:
# DEFININDO COMANDO PARA AVALIAR O DESEMPENHO DO MODELO NAS IMAGENS DE TESTE #

test_loss, test_acc = model.evaluate(test_data) # verificar versão do Keras

print("acurácia nos dados de teste:", test_acc)
print("loss dos dados de teste:", test_loss)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score, accuracy_score

Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis=1)

print(y_pred.shape)

print(test_data.classes)

print('Accuracy')
print(accuracy_score(test_data.classes,y_pred))

print('Balanced Accuracy')
print(balanced_accuracy_score(test_data.classes,y_pred))

print('Confusion Matrix')
print(confusion_matrix(test_data.classes, y_pred))
print('Classification Report')

In [ ]:
model.summary()